In [ ]:
# Cell 0: Setup
import os, subprocess
REPO = "/kaggle/working/competition_package"
os.chdir("/kaggle/working")
os.system(f"rm -rf {REPO}")
os.system(f"git clone https://github.com/vincentvdo6/competition_package.git {REPO}")
os.chdir(REPO)
os.makedirs("datasets", exist_ok=True)
os.makedirs("logs", exist_ok=True)
os.system("ln -sf /kaggle/input/wunderfund-predictorium/train.parquet datasets/train.parquet")
os.system("ln -sf /kaggle/input/wunderfund-predictorium/valid.parquet datasets/valid.parquet")
print("Commit:", subprocess.check_output(["git", "rev-parse", "--short", "HEAD"], text=True).strip())
print("Ready!")

In [ ]:
# Cell 1: Train 7x GRU tightwd_v2 combined-loss (seeds 47-53)
# ~95s each x 7 = ~11 min
import os
os.chdir("/kaggle/working/competition_package")
for seed in [47, 48, 49, 50, 51, 52, 53]:
    print(f"\n{'='*60}")
    print(f"Training gru_derived_tightwd_v2 seed {seed}")
    print(f"{'='*60}")
    os.system(
        f"python -u scripts/train.py "
        f"--config configs/gru_derived_tightwd_v2.yaml "
        f"--seed {seed} --device cuda"
    )

In [ ]:
# Cell 2: Train 6x GRU pearson_v1 (seeds 45-50)
# ~95s each x 6 = ~10 min
import os
os.chdir("/kaggle/working/competition_package")
for seed in [45, 46, 47, 48, 49, 50]:
    print(f"\n{'='*60}")
    print(f"Training gru_pearson_v1 seed {seed}")
    print(f"{'='*60}")
    os.system(
        f"python -u scripts/train.py "
        f"--config configs/gru_pearson_v1.yaml "
        f"--seed {seed} --device cuda"
    )

In [ ]:
# Cell 3: Strip checkpoints and copy normalizers
import os, torch, glob, shutil
os.chdir("/kaggle/working/competition_package")
os.makedirs("logs/slim", exist_ok=True)

for pt in sorted(glob.glob("logs/*.pt")):
    try:
        ckpt = torch.load(pt, map_location="cpu", weights_only=False)
    except TypeError:
        ckpt = torch.load(pt, map_location="cpu")
    slim = {"model_state_dict": ckpt["model_state_dict"], "config": ckpt.get("config", {})}
    out = f"logs/slim/{os.path.basename(pt)}"
    torch.save(slim, out)
    orig = os.path.getsize(pt) / 1e6
    new = os.path.getsize(out) / 1e6
    print(f"{os.path.basename(pt)}: {orig:.1f}MB -> {new:.1f}MB")

for npz in sorted(glob.glob("logs/normalizer_*.npz")):
    shutil.copy(npz, f"logs/slim/{os.path.basename(npz)}")
    print(f"Copied {os.path.basename(npz)}")

print(f"\n--- logs/slim/ contents ---")
for f in sorted(os.listdir("logs/slim")):
    sz = os.path.getsize(f"logs/slim/{f}") / 1e6
    print(f"  {f}: {sz:.1f}MB")

In [ ]:
# Cell 4: Zip for download
import shutil, os
shutil.make_archive("/kaggle/working/gru_seed_expansion", "zip",
                     "/kaggle/working/competition_package/logs/slim")
sz = os.path.getsize("/kaggle/working/gru_seed_expansion.zip") / 1e6
print(f"gru_seed_expansion.zip: {sz:.1f}MB")
print("Download from: /kaggle/working/gru_seed_expansion.zip")

In [ ]:
# Cell 5: Print validation scores
import os, glob, torch
os.chdir("/kaggle/working/competition_package")
print(f"{'Model':<50} {'Best Val':>10}")
print("-" * 62)
for pt in sorted(glob.glob("logs/*.pt")):
    try:
        ckpt = torch.load(pt, map_location="cpu", weights_only=False)
    except TypeError:
        ckpt = torch.load(pt, map_location="cpu")
    score = ckpt.get("best_score", ckpt.get("val_score", "N/A"))
    name = os.path.basename(pt)
    if isinstance(score, float):
        print(f"{name:<50} {score:>10.4f}")
    else:
        print(f"{name:<50} {str(score):>10}")